# **로그 분석을 통한 보안 위험도 예측 AI 경진대회**
-Team SsulleBal
https://dacon.io/competitions/official/235717/codeshare/2679?page=2&dtype=recent

- 기존에 확인된 위험요소는 0-6의 위험도 분류, 새로운 위험 요소는 위험도 7로 분류
- Level 7 구별


- 로그 데이터 전처리 > Validation을 통해 등급별로 threshold 상세 설정
- 실제 서비스에서 Line by Line으로 들어왔을 때의 분류방법:  
  1) 로그 문장 데이터 전처리 진행 (Cleaning)  
  2) train 데이터와 완전히 동일한 문장이 있는지 확인하여 서로 다른 모델 적용(A방법, B방법)  
  -A방법: 동일한 문장이 있다면 0-6 등급 지도 학습된 모델로 Predict(Threshold 적용 X)  
  -B방법: 동일한 문장이 없다면 0-6 등급 지도 학습된 모델로 Predict 이후에 설정된 Threshold 를 통해 7은 따로 분류  
  3) B방법에서 Threshold를 통해 걸러지지 않는다면 A방법과 동일한 모델로 predict 적용

1> 로그 문장 데이터 전처리 진행(Cleaning)   

2> CountVectorizer로 feature 추출 > RandomForestClassifier
- train 데이터와 동일한 문장이 있다면, 0-6 등급 지도 학습된 모델로 Predict
- 전처리 덜 된 데이터로 모델 생성
- 동일한 문장이 없다면, 모델 Predict 후 Validation 데이터로 설정한 Threshold를 통해 위험도 7 분류

### **1. Data&Library IMPORT**

In [1]:
conda install dask

Solving environment: ...working... done

## Package Plan ##

Note: you may need to restart the kernel to use updated packages.



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3





  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - dask


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.13.0               |   py38haa95532_0         926 KB
    ------------------------------------------------------------
                                           Total:         926 KB

The following packages will be UPDATED:

  conda              conda-forge::conda-4.10.3-py38haa244f~ --> pkgs/main::conda-4.13.0-py38haa95532_0




conda-4.13.0         | 926 KB    |            |   0% 
conda-4.13.0         | 926 KB    | #5         |  16% 
conda-4.13.0         | 926 KB    | ########4  |  85% 
conda-4.13.0         | 926 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... failed


In [2]:
import pandas as pd
import numpy as np
import os 
import pandas as pds
from dask import dataframe
import re
import numpy as np
import seaborn as sbn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet

In [3]:
train = pd.read_csv("train.csv")
test= pd.read_csv("test.csv")

In [10]:
train.head()

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...


In [5]:
test.head()

,id,full_log
0,1000000,"Feb 8 15:47:26 localhost kibana: {""type"":""err..."
1,1000001,"Sep 24 03:46:39 localhost kibana: {""type"":""err..."
2,1000002,type=SYSCALL msg=audit(1611888200.428:210563):...
3,1000003,"Jan 18 11:24:06 localhost kibana: {""type"":""err..."
4,1000004,type=SYSCALL msg=audit(1603081202.050:46851): ...


In [6]:
len(test)

1418916

### **2. EDA**

1) 기호 및 숫자 제거
- re.sub(정규표현식,'',변경하고 싶은 문자): 정규식을 이용해 특정 길이 단어 제거
- '\s' : 띄어쓰기, 탭, 줄바꿈 같은 공간

In [11]:
lit = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

def mask(tt):
    tt=tt.apply(lambda x: re.sub(r'(\\n)',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+',' ',x))
    #tt=tt.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
    tt=tt.apply(lambda x: re.sub(r'[0-9]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r"':/()",' ',x))
    tt=tt.apply(lambda x: re.sub(r':',' ',x))
    tt=tt.apply(lambda x: re.sub(r',',' ',x))
    # = tt.apply(lambda x: re.sub(r'(',' ',x))
    #t = tt.apply(lambda x: re.sub(r')',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',x))
    
    for st in lit:
        st = " "+ st + " "
        tt=tt.apply(lambda x: re.sub(st,' ',x))
        
    tt=tt.apply(lambda x: re.sub(r'\s+',' ',x))
    
    return tt

In [12]:
train['pre_log'] = mask(train.full_log)
test['pre_log'] = mask(test.full_log)

In [13]:
train.head()

,id,level,full_log,pre_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err...",Sep localhost kibana type error timestamp tags...
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...,Feb localhost logstash INFO logstash outputs e...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp tags...
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp tags...
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...,type SYSCALL msg audit arch syscall success ye...


2) 영어 단어가 아니거나 3글자 미만인 경우 삭제
- Validation 7등급을 보았을 때, 의미가 있는 영어 단어들이 새로 들어오는 경향이 있다고 판단

In [14]:
#포함되어 있는 단어 모음 생성
def count_word(data):
    tem = list(data['pre_log'].str.split(" "))
    all_word = []
    for word in tem:
        all_word.extend(word)
    words = pd.Series(all_word)
    return words.value_counts()

In [15]:
train_count = count_word(train)
test_count = count_word(test)

- set: 집합  
    -중복 제거

In [16]:
train_words = list(set(train_count.index))
test_words = list(set(test_count.index))

In [17]:
#영어 단어 + 3글자 이상인 경우만 True
#check_words는 단어인지를 확인해주고 결과를 t/f로 반환

from nltk.corpus import words

def check_words(word_list: list):
    re = [False] * len(word_list)
    
    for i,word in enumerate(word_list):
        if len(word) < 3:
            continue
        word = word.lower()
        if word in words.words():
            re[i] = True
            
    return re

In [18]:
word_list = ['Promiscuous', 'ab', 'nihongo', 'abstract', 'pedo','gid']
check_words(word_list)

[True, False, False, True, False, True]

In [19]:
train_tf = check_words(train_words)

In [20]:
test_tf = check_words(test_words)

- train_words: 포함되어 있는 단어 모음    
  train_tf=check_words(train_words) T/F  
  dic[train_words]=train_tf

In [ ]:
#단어인지 확인해주는 딕셔너리 만들어주기 (train_isword,test_isword)
train_isword = dict()
test_isword = dict()

total_words = list(train_words)
test_words = list(test_words)

for i in range(len(train_words)):
    train_isword[train_words[i].lower()] = train_tf[i]
    
for i in range(len(test_words)):
    test_isword[test_words[i].lower()] = test_tf[i]

3) Train, Test에 각각 적용
- 미리 사용된 단어들을 모아서 딕셔너리를 만들어서 사용하며 시간 단축
- Data Leakage 가능성

In [ ]:
#단어를 하나씩 확인하는 방법(60~70 시간 이상 소요) 
from nltk.corpus import words 

def check_words(word_list: list): 
    re = [False] * len(word_list) 
    for i,word in enumerate(word_list): 
        word = word.lower() 
        if len(word)>2: 
            if word in words.words(): 
                re[i] = True 
    return re

def cutt(data): 
    data = data.lower() 
    splited = data.split(" ") 
    check = check_words(splited) 
    c = np.array(splited) 
    real_words = list(c[check]) 
    tem = " ".join(real_words) 
    return tem

#valid['cut'] = valid['pre_log'].map(cutt) 
#train['cut'] =train['pre_log'].map(cutt) 
#test['cut'] =test['pre_log'].map(cutt)

In [ ]:
import copy

def checking_train(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if train_isword[word]:
            re[i] = True
    return re

def checking_test(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if test_isword[word]:
            re[i] = True
    return re

def cutt_train(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_train(splited)
    c = np.array(splited)
    real_words = list(c[check])    
    tem = " ".join(real_words)
    #tem = tem.lower() 
    return tem

def cutt_test(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_test(splited)
    c = np.array(splited)
    real_words = list(c[check])  
    tem = " ".join(real_words)
    #tem = tem.lower()
    return tem

- map 함수 이용

In [ ]:
train['cut'] = train['pre_log'].map(cutt_train)
test['cut'] = test['pre_log'].map(cutt_test)

In [ ]:
del train['full_log']
del test['full_log']

del train['pre_log']
del test['pre_log']

4) 나중에 사용될 inornot 리스트
- train 데이터에 100% 동일한 문장이 있는지 확인

In [ ]:
train_bag = train['cut'].unique()
len(train_bag)

In [ ]:
#inornot을 통해서 train안에 같은 문장이 있으면 0 없으면 1
from tqdm import tqdm

inornot = np.zeros(len(test))

for i in tqdm(range(len(test))):
    tem = test.iloc[i]['cut']
    if tem not in train_bag:
        inornot[i] = 1

### **3. 모델링**
- CountVectorizer: 각 문서에서 해당 단어가 나타나는 횟수를 Count하여 피처로 사용  
→ train/test_features


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

#남은 단어가 없는 경우에는 'missing' 으로 대체
train['cut'] = train['cut'].replace('','missing',regex=True)
test['cut'] = test['cut'].replace('','missing',regex=True)

train = train.fillna("missing")
test = test.fillna("missing")

In [ ]:
train_text=list(train['cut'])
train_level=np.array(train['level'])

test_text=list(test['cut'])

In [ ]:
vectorizer=CountVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)- RandomForestClassifier 사용

- RandomForestClassifier 사용

In [ ]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

In [ ]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

- Validation을 이용해 위험도 7 분류  

1) 모델 학습 후 test셋 예측  
2) test에서 유추할 수 있는 level7의 예측확률 확인  
3) 예측확률에 맞게 threshold 조정 ( [0.37<preds<0.3876] = level7) 

In [ ]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 1))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 2))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.95) & (results == 3))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 4))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 5))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 6))[0]]=7

In [ ]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.94001) & (np.max(results_proba, axis=1) > 0.93999) & (results == 1))[0]]=7 
results[np.where((np.max(results_proba, axis=1)<0.611657) & (np.max(results_proba, axis=1) > 0.6116568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.571657) & (np.max(results_proba, axis=1) > 0.5716568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.68001) & (np.max(results_proba, axis=1) > 0.67999) & (results == 5))[0]]=7 

### **4. 결과 저장**

In [ ]:
submission=pd.read_csv('sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

In [ ]:
submission.to_csv("tem_answer.csv",index = False)

### **5. 추가 작업**
- test 데이터에 동일한 문장이 있는지 확인하는 컬럼 추가
- 같은 문장이 있으면 0 없으면 1

In [ ]:
submission['inornot'] = pd.Series(inornot)

In [ ]:
cal = submission[(submission['level']==7) & (submission['inornot'] == 0)].index #위험도 7 & 같은 문장 있는 경우
not7_id = list(test.iloc[cal]['id'])
pd.Series(not7_id).to_csv("not7_id.csv",index= False)

- 7를 제외한 나머지 등급에는 전처리가 덜 진행된 Data로 지도학습 분류

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train['full_log']=train['full_log'].str.replace(r'[0-9]', '<num>')
test['full_log']=test['full_log'].str.replace(r'[0-9]', '<num>')

In [ ]:
train = train.fillna("missing")
test = test.fillna("missing")

In [ ]:
train_text=list(train['full_log'])
train_level=np.array(train['level'])

test_text=list(test['full_log'])
#ids=list(test['id'])

In [ ]:
vectorizer=TfidfVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [ ]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [ ]:
submission=pd.read_csv('sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

In [ ]:
not7_id = pd.read_csv('not7_id.csv')
not7_id = list(not7_id['0'])

- 위에서 잡은 7 합치기 & train에 있는 경우는 사용하지 않기

In [ ]:
only = pd.read_csv("tem_answer.csv")

le7 = list(only[only['level']==7]['id'])
le7_update = list((set(le7))-set(not7_id))
print(len(le7))
print(len(le7_update))

idx = submission[submission['id'].isin(le7_update)].index
submission.iloc[idx] = 7
submission['level'].value_counts()

In [ ]:
who = submission.level
submission=pd.read_csv('sample_submission.csv')
submission['level'] = who
submission['level'].value_counts()

In [ ]:
submission.to_csv("581_final_answer.csv",index = False)